<a href="https://colab.research.google.com/github/luiz-felipe-moreira/car-damage-segmentation/blob/main/images_to_tfrecord.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instance Segmentation de Danos em Automóveis usando TF Model Garden


Este notebook faz o treinamento (fine-tuning) de uma [Mask R-CNN](https://arxiv.org/abs/1703.06870) implementada no [pacote TensorFlow Model Garden](https://pypi.org/project/tf-models-official/) (tensorflow-models). O backbone utilizado (checkpoint) também foi obtido do [Model Garden](https://www.tensorflow.org/tfmodels).  
  
Descrição do Model Garden:  
"[Model Garden](https://www.tensorflow.org/tfmodels) contains a collection of state-of-the-art models, implemented with TensorFlow's high-level APIs. The implementations demonstrate the best practices for modeling, letting users to take full advantage of TensorFlow for their research and product development."

##Verifica se estamos usando GPU e qual sua configuração

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Nov 26 16:26:55 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    25W / 300W |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 54.8 gigabytes of available RAM

You are using a high-RAM runtime!


## Instala as dependências

In [ ]:
!python --version

Python 3.10.12


In [ ]:
!pip install -q "tf-models-official"==2.14.2
!pip install -U -q remotezip tqdm opencv-python einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.4/106.4 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 19.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 MB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 5.6 MB/s eta 0:00:00


## Importa as bibliotecas

In [ ]:
import os
import io
import json
import tqdm
import shutil
import pprint
import pathlib
import tempfile
import requests
import collections
import matplotlib
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from PIL import Image
from six import BytesIO
from etils import epath
from IPython import display
from urllib.request import urlopen

In [ ]:
import orbit
import tensorflow as tf
import tensorflow_models as tfm
import tensorflow_datasets as tfds

from official.core import exp_factory, base_trainer
from official.core import config_definitions as cfg
from official.vision.data import tfrecord_lib
from official.vision.serving import export_saved_model_lib
from official.vision.dataloaders.tf_example_decoder import TfExampleDecoder
from official.vision.utils.object_detection import visualization_utils
from official.vision.ops.preprocess_ops import normalize_image, resize_and_crop_image
from official.vision.data.create_coco_tf_record import coco_annotations_to_lists

pp = pprint.PrettyPrinter(indent=4) # Set Pretty Print Indentation
print(tf.__version__) # Check the version of tensorflow used

%matplotlib inline

2.14.0


## Definição da quantidade de classes do dataset

In [ ]:
NUM_CLASSES = 6

## Download dataset e conversão para TFRecord
Faz download do dataset (imagens do meu Google Drive e anotações do Cloud Storage) para a máquina usada pelo Colab.

In [ ]:
# @title Download JSONs de anotação

!gsutil cp gs://car-damage-recognition/instances_train2017.json .
!gsutil cp gs://car-damage-recognition/instances_val2017.json .
!gsutil cp gs://car-damage-recognition/instances_test2017.json .

Copying gs://car-damage-recognition/instances_train2017.json...
| [1 files][ 19.6 MiB/ 19.6 MiB]                                                
Operation completed over 1 objects/19.6 MiB.                                     
Copying gs://car-damage-recognition/instances_val2017.json...
- [1 files][  5.7 MiB/  5.7 MiB]                                                
Operation completed over 1 objects/5.7 MiB.                                      
Copying gs://car-damage-recognition/instances_test2017.json...
- [1 files][  2.5 MiB/  2.5 MiB]                                                
Operation completed over 1 objects/2.5 MiB.                                      


In [ ]:
# @title Funções para parsing das anotações do dataset CarDD

def get_category_map(annotation_path, num_classes):
  with epath.Path(annotation_path).open() as f:
      data = json.load(f)

  category_map = {id+1: {'id': cat_dict['id'],
                       'name': cat_dict['name']}
                  for id, cat_dict in enumerate(data['categories'][:num_classes])}
  return category_map

class CarDDAnnotation:
  """CarDD annotation helper class.
  """

  def __init__(self, annotation_path):
    with epath.Path(annotation_path).open() as f:
      data = json.load(f)
    self._data = data

    img_id2annotations = collections.defaultdict(list)
    for a in self._data.get('annotations', []):
      if a['category_id'] in category_ids:
        img_id2annotations[a['image_id']].append(a)
    self._img_id2annotations = {
        k: list(sorted(v, key=lambda a: a['id']))
        for k, v in img_id2annotations.items()
    }

  @property
  def categories(self):
    """Return the category dicts, as sorted in the file."""
    return self._data['categories']

  @property
  def images(self):
    """Return the image dicts, as sorted in the file."""
    sub_images = []
    for image_info in self._data['images']:
      if image_info['id'] in self._img_id2annotations:
        sub_images.append(image_info)
    return sub_images

  def get_annotations(self, img_id):
    """Return all annotations associated with the image id string."""
    # Some images don't have any annotations. Return empty list instead.
    return self._img_id2annotations.get(img_id, [])

In [ ]:
# @title Obtenção das categorias
train_annotation_path = './instances_train2017.json'
valid_annotation_path = './instances_val2017.json'
test_annotation_path = './instances_test2017.json'

category_index = get_category_map(valid_annotation_path, NUM_CLASSES)
category_ids = list(category_index.keys())
category_index

{1: {'id': 1, 'name': 'dent'},
 2: {'id': 2, 'name': 'scratch'},
 3: {'id': 3, 'name': 'crack'},
 4: {'id': 4, 'name': 'glass shatter'},
 5: {'id': 5, 'name': 'lamp broken'},
 6: {'id': 6, 'name': 'tire flat'}}

In [ ]:
# @title Download das imagens do dataset, salvas no Google Drive

from google.colab import drive
drive.mount('/content/drive')

if not os.path.exists('./cardd_dataset/'):
  os.mkdir('./cardd_dataset/')
!cp -r /content/drive/My\ Drive/instance_segmentation_data/CarDD_COCO/train2017 /content/cardd_dataset
!cp -r /content/drive/My\ Drive/instance_segmentation_data/CarDD_COCO/val2017 /content/cardd_dataset
!cp -r /content/drive/My\ Drive/instance_segmentation_data/CarDD_COCO/test2017 /content/cardd_dataset

Mounted at /content/drive


##Geração dos TFRecords

In [ ]:
def _generate_tf_records(prefix, images_dir, annotation_file, output_dir, num_shards=8):
    """Generate TFRecords."""

    cardd_annotation = CarDDAnnotation(annotation_file)

    def _process_example(prefix, image_info, id_to_name_map):
      filename = image_info['file_name']

      image = tf.io.read_file(os.path.join(images_dir, filename))
      instances = cardd_annotation.get_annotations(img_id=image_info['id'])

      data, _ = coco_annotations_to_lists(instances,
                                          id_to_name_map,
                                          image_info['height'],
                                          image_info['width'],
                                          include_masks=True)
      keys_to_features = {
          'image/encoded':
              tfrecord_lib.convert_to_feature(image.numpy()),
          'image/filename':
               tfrecord_lib.convert_to_feature(filename.encode('utf8')),
          'image/format':
              tfrecord_lib.convert_to_feature('jpg'.encode('utf8')),
          'image/height':
              tfrecord_lib.convert_to_feature(image_info['height']),
          'image/width':
              tfrecord_lib.convert_to_feature(image_info['width']),
          'image/source_id':
              tfrecord_lib.convert_to_feature(str(image_info['id']).encode('utf8')),
          'image/object/bbox/xmin':
              tfrecord_lib.convert_to_feature(data['xmin']),
          'image/object/bbox/xmax':
              tfrecord_lib.convert_to_feature(data['xmax']),
          'image/object/bbox/ymin':
              tfrecord_lib.convert_to_feature(data['ymin']),
          'image/object/bbox/ymax':
              tfrecord_lib.convert_to_feature(data['ymax']),
          'image/object/class/text':
              tfrecord_lib.convert_to_feature(data['category_names']),
          'image/object/class/label':
              tfrecord_lib.convert_to_feature(data['category_id']),
          'image/object/is_crowd':
              tfrecord_lib.convert_to_feature(data['is_crowd']),
          'image/object/area':
              tfrecord_lib.convert_to_feature(data['area'], 'float_list'),
          'image/object/mask':
              tfrecord_lib.convert_to_feature(data['encoded_mask_png'])
      }
      example = tf.train.Example(
          features=tf.train.Features(feature=keys_to_features))
      return example

    writers = [
        tf.io.TFRecordWriter(
            output_dir + prefix +'-%05d-of-%05d.tfrecord' % (i, num_shards))
        for i in range(num_shards)
    ]
    id_to_name_map = {cat_dict['id']: cat_dict['name']
                      for cat_dict in cardd_annotation.categories[:NUM_CLASSES]}
    for idx, image_info in enumerate(tqdm.tqdm(cardd_annotation.images)):
      tf_example = _process_example(prefix, image_info, id_to_name_map)
      writers[idx % num_shards].write(tf_example.SerializeToString())

    del cardd_annotation

In [ ]:
train_prefix = 'train'
valid_prefix = 'val'
test_prefix = 'test'

TRAIN_IMGS_DIR = './cardd_dataset/train2017/'
VALID_IMGS_DIR = './cardd_dataset/val2017/'
TEST_IMGS_DIR = './cardd_dataset/test2017/'


tf_records_dir = './cardd_tfrecords_all_sets/'
if not os.path.exists(tf_records_dir):
  os.mkdir(tf_records_dir)

_generate_tf_records(train_prefix,
                     TRAIN_IMGS_DIR,
                     train_annotation_path,
                     tf_records_dir,
                     16)

_generate_tf_records(valid_prefix,
                     VALID_IMGS_DIR,
                     valid_annotation_path,
                     tf_records_dir,
                     8)

_generate_tf_records(test_prefix,
                     TEST_IMGS_DIR,
                     test_annotation_path,
                     tf_records_dir,
                     4)

Copia as pastas contendo os arquivos TFRecords de cada conjunto para um bucket do Google Cloud Storage. Os arquivos ficarão disponíveis no GCS no seguinte caminho:





*   Treino: gs://car-damage-recognition/cardd_tfrecords_all_sets/train
*   Validação: gs://car-damage-recognition/cardd_tfrecords_all_sets/val
*   Teste: gs://car-damage-recognition/cardd_tfrecords_all_sets/test

In [ ]:
!gsutil cp -r './cardd_tfrecords_all_sets' gs://car-damage-recognition

Copying file://./cardd_tfrecords_all_sets/val-00001-of-00008.tfrecord [Content-Type=application/octet-stream]...
Copying file://./cardd_tfrecords_all_sets/train-00013-of-00016.tfrecord [Content-Type=application/octet-stream]...
Copying file://./cardd_tfrecords_all_sets/train-00006-of-00016.tfrecord [Content-Type=application/octet-stream]...
Copying file://./cardd_tfrecords_all_sets/train-00014-of-00016.tfrecord [Content-Type=application/octet-stream]...
\ [4 files][459.5 MiB/459.5 MiB]   28.0 MiB/s                                   
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying file://./cardd_tfrecords_all_sets/train-00005-of-00016.tfrecord [Content-Type=application/octet-stream]...
Copying file://./cardd_tfrecords_all_sets/train-00003-of-00016.tfrecord [Content-Type=applicat